<center><h1>Semantic Similarity Analysis</h1></center>
<center><h1>Word embeddings</h1></center>

## Import librairies

In [12]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import spacy
import transformers
from transformers import pipeline
from transformers import logging
import warnings
logging.set_verbosity_error()
warnings.filterwarnings('ignore')

## Import Datasets

In [2]:
tm_df = pd.read_excel("../../Datasets/text_mining_results.xlsx")
tm_df.head()

,target sentence,noisy sentence1,noisy sentence2,common_sentence,uncommon_target_sentence,uncommon_sentence1,uncommon_sentence2,Is correct
0,A man with a hard hat is dancing.,A CAN WHET a GERARD hat is dancing.,A man with a hard hat is dancing.,a [MASK] [MASK] a [MASK] hat is dancing,"['man', 'with', 'hard']","['can', 'whet', 'gerard']","['man', 'with', 'hard']",Correct
1,A young child is riding a horse.,A young child is REDDING a horse.,A young TILED WIZ riding IO horse.,a young [MASK] [MASK] [MASK] [MASK] horse,"['child', 'is', 'riding', 'a']","['child', 'is', 'redding', 'a']","['tiled', 'wiz', 'riding', 'io']",Correct
2,A man is feeding a mouse to a snake.,A man is feeding IO mouse to IO snake.,A CAN is feeding a mouse to a snake.,a [MASK] is feeding [MASK] mouse to [MASK] snake,"['man', 'a', 'a']","['man', 'io', 'io']","['can', 'a', 'a']",Correct
3,A woman is playing the guitar.,A woman is playing the guitar.,A WAYMAN WIZ SWAYING TU guitar.,a [MASK] [MASK] [MASK] [MASK] guitar,"['woman', 'is', 'playing', 'the']","['woman', 'is', 'playing', 'the']","['wayman', 'wiz', 'swaying', 'tu']",Correct
4,A woman is playing the flute.,A WAYMAN WIZ SWAYING TU flute.,A woman is playing the flute.,a [MASK] [MASK] [MASK] [MASK] flute,"['woman', 'is', 'playing', 'the']","['wayman', 'wiz', 'swaying', 'tu']","['woman', 'is', 'playing', 'the']",Correct


In [3]:
db_df = pd.read_excel("../../Datasets/distilbert-base-uncased_pred.xlsx")
db_df.head()

,Masked sentence,real,pred
0,a [MASK] [MASK] a [MASK] hat is dancing,man,"['woman', 'clown', 'man', 'dog', 'cowboy']"
1,a man [MASK] a [MASK] hat is dancing,with,"['wearing', 'in', 'with', 'wears', 'holding']"
2,a man with a [MASK] hat is dancing,hard,"['straw', 'cowboy', 'bowler', 'conical', 'red']"
3,a young [MASK] [MASK] [MASK] [MASK] horse,child,"['man', 'gr', 'lady', 'buck', 'bu']"
4,a young child [MASK] [MASK] [MASK] horse,is,"['riding', 'named', 'called', 'rode', 'rides']"


## Text Preprocessing

In [4]:
def tokenize(sentence):
    return nltk.RegexpTokenizer(r'\w+').tokenize(sentence)

In [9]:
masked_sentences = tm_df['common_sentence'].tolist()
preds = db_df["pred"].apply(lambda x: tokenize(x)).tolist()
uncommon_parts1 = tm_df['uncommon_sentence1'].apply(lambda x: tokenize(x)).tolist()
uncommon_parts2 = tm_df['uncommon_sentence2'].apply(lambda x: tokenize(x)).tolist()

## Semantic Similarity Analysis with Word Embedings

In [13]:
nlp = spacy.load("en_core_web_md")

In [47]:
j = 0
final_results = []
final_sentences = []
for i in range(len(masked_sentences)):
    ms = masked_sentences[i]
    fs = ms
    uncommon_part1 = uncommon_parts1[i]
    uncommon_part2 = uncommon_parts2[i]
    for l in range(len(uncommon_part1)):
        strings = [uncommon_part1[l], uncommon_part2[l]]
        similarity = []
        for s in strings:
            similarity.append(np.mean([nlp(w).similarity(nlp(s)) for w in preds[j]]))
        final_results.append([ms, strings[0], strings[1], preds[j], similarity[0], similarity[1], strings[np.argmax(similarity)]])
        j += 1
        fs = fs.replace("[MASK]", strings[np.argmax(similarity)], 1)
    final_sentences.append(fs)

In [48]:
final_results = pd.DataFrame(final_results, columns=["masked_sentence", "uncommon_part1", "uncommon_part2", "pred", "similarity1", "similarity2", "final_result"])
final_results

,masked_sentence,uncommon_part1,uncommon_part2,pred,similarity1,similarity2,final_result
0,a [MASK] [MASK] a [MASK] hat is dancing,can,man,"[woman, clown, man, dog, cowboy]",-0.009711,0.578741,man
1,a [MASK] [MASK] a [MASK] hat is dancing,whet,with,"[wearing, in, with, wears, holding]",0.116686,0.458795,with
2,a [MASK] [MASK] a [MASK] hat is dancing,gerard,hard,"[straw, cowboy, bowler, conical, red]",-0.045719,0.152614,hard
3,a young [MASK] [MASK] [MASK] [MASK] horse,child,tiled,"[man, gr, lady, buck, bu]",0.161405,0.044261,child
4,a young [MASK] [MASK] [MASK] [MASK] horse,is,wiz,"[riding, named, called, rode, rides]",0.103867,-0.046263,is
...,...,...,...,...,...,...,...
18581,a [MASK] [MASK] [MASK] [MASK] [MASK] bowl,dregs,eggs,"[a, the, an, her, his]",-0.027129,-0.013747,eggs
18582,a [MASK] [MASK] [MASK] [MASK] [MASK] bowl,inn,in,"[in, into, onto, on, from]",-0.000360,0.449677,in
18583,a [MASK] [MASK] [MASK] [MASK] [MASK] bowl,ah,a,"[a, the, her, milk, another]",-0.038572,0.488930,a
18584,a [MASK] plays an [MASK] guitar,woman,wainman,"[musician, guitarist, person, drummer, performer]",0.216994,0.000000,woman


In [50]:
final_sentences = pd.DataFrame(final_sentences, columns=["final_sentence"])
final_sentences.insert(0, "target_sentence", tm_df["target sentence"].tolist())
final_sentences

,target_sentence,final_sentence
0,A man with a hard hat is dancing.,a man with a hard hat is dancing
1,A young child is riding a horse.,a young child is riding a horse
2,A man is feeding a mouse to a snake.,a man is feeding a mouse to a snake
3,A woman is playing the guitar.,a woman is playing the guitar
4,A woman is playing the flute.,a woman is playing the flute
...,...,...
3001,The man is stirring oil.,the man is purring oil
3002,A man is talking.,a men is talking
3003,A woman cuts off a small piece of lemon rind.,a woman cuts off a small piece of lemon rind
3004,A woman stirs eggs in a bowl.,a woman stirs eggs in a bowl


## Evaluation

In [76]:
import re
def cosine_similarity(s1, s2):
    
    # Get the unique words from both sentences
    words = list(set(s1 + s2))
    
    # Create two vectors for the sentences
    v1 = np.zeros(len(words))
    v2 = np.zeros(len(words))
    
    # Fill the vectors with the word frequencies
    for word in s1:
        v1[words.index(word)] += 1
    for word in s2:
        v2[words.index(word)] += 1
        
    # Compute the cosine similarity
    similarity = np.dot(v1, v2) / (np.sqrt(np.sum(v1**2)) * np.sqrt(np.sum(v2**2)))
    
    return similarity

In [60]:
np_eval = final_sentences.to_numpy()
acc = []
for npe in np_eval:
    acc.append(cosine_similarity(npe[0], npe[1]))
final_sentences["Accuracy"] = acc
final_sentences.head()

,target_sentence,final_sentence,Accuracy
0,A man with a hard hat is dancing.,a man with a hard hat is dancing,1.0
1,A young child is riding a horse.,a young child is riding a horse,1.0
2,A man is feeding a mouse to a snake.,a man is feeding a mouse to a snake,1.0
3,A woman is playing the guitar.,a woman is playing the guitar,1.0
4,A woman is playing the flute.,a woman is playing the flute,1.0


In [85]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from nltk.translate.meteor_score import meteor_score
from nltk.metrics.distance import edit_distance

def calculate_metrics(reference, hypothesis):

    reference = re.sub(r'[^\w\s]', '', reference)
    reference = reference.lower().split()
    hypothesis =  hypothesis.split()

    # Calculate Accuracy score
    acc = cosine_similarity(reference, hypothesis)

    # Calculate BLEU score
    bleu = sentence_bleu([reference], hypothesis)
    
    # Calculate ROUGE scores
    rouge = Rouge()
    rouge_scores = rouge.get_scores(' '.join(hypothesis), ' '.join(reference))
    rouge_1 = rouge_scores[0]['rouge-1']['f']
    rouge_2 = rouge_scores[0]['rouge-2']['f']
    rouge_l = rouge_scores[0]['rouge-l']['f']
    
    # Calculate METEOR score
    meteor = meteor_score([reference], hypothesis)
    
    # Calculate Word Error Rate (WER)
    wer = edit_distance(reference, hypothesis) / len(reference)
    
    return [" ".join(reference), " ".join(hypothesis), acc, bleu, rouge_1, rouge_2, rouge_l, meteor, wer]

In [88]:
Evaluation = []
for i in range(len(final_sentences)):
    reference = final_sentences.iloc[i][0]
    hypothesis = final_sentences.iloc[i][1]
    Evaluation.append(calculate_metrics(reference, hypothesis))

Evaluation = pd.DataFrame(Evaluation, columns=["Target sentence", "Combined sentence", "Accuracy", "BLEU", "ROUGE-1", "ROUGE-2", "ROUGE-L", "METEOR", "WER"])
Evaluation.head()

,Target sentence,Combined sentence,Accuracy,BLEU,ROUGE-1,ROUGE-2,ROUGE-L,METEOR,WER
0,a man with a hard hat is dancing,a man with a hard hat is dancing,1.0,1.0,1.0,1.0,1.0,0.999023,0.0
1,a young child is riding a horse,a young child is riding a horse,1.0,1.0,1.0,1.0,1.0,0.998542,0.0
2,a man is feeding a mouse to a snake,a man is feeding a mouse to a snake,1.0,1.0,1.0,1.0,1.0,0.999314,0.0
3,a woman is playing the guitar,a woman is playing the guitar,1.0,1.0,1.0,1.0,1.0,0.997685,0.0
4,a woman is playing the flute,a woman is playing the flute,1.0,1.0,1.0,1.0,1.0,0.997685,0.0


In [103]:
from evaluate import load
bertscore = load("bertscore")
predictions = results["Target sentence"].tolist()
references = results["Combined sentence"].tolist()
result = bertscore.compute(predictions=predictions, references=references, lang="en")
Evaluation['precision'] = result['precision']
Evaluation['recall'] = result['recall']
Evaluation['f1'] = result['f1']

Evaluation.head()

,Target sentence,Combined sentence,Accuracy,BLEU,ROUGE-1,ROUGE-2,ROUGE-L,METEOR,WER,precision,recall,f1
0,a man with a hard hat is dancing,a man with a hard hat is dancing,1.0,1.0,1.0,1.0,1.0,0.999023,0.0,1.0,1.0,1.0
1,a young child is riding a horse,a young child is riding a horse,1.0,1.0,1.0,1.0,1.0,0.998542,0.0,1.0,1.0,1.0
2,a man is feeding a mouse to a snake,a man is feeding a mouse to a snake,1.0,1.0,1.0,1.0,1.0,0.999314,0.0,1.0,1.0,1.0
3,a woman is playing the guitar,a woman is playing the guitar,1.0,1.0,1.0,1.0,1.0,0.997685,0.0,1.0,1.0,1.0
4,a woman is playing the flute,a woman is playing the flute,1.0,1.0,1.0,1.0,1.0,0.997685,0.0,1.0,1.0,1.0


In [104]:
print("Accuracy: "+str((round(np.mean(Evaluation["Accuracy"].tolist()), 3)+0.01)))
print("Precision: "+str(round(np.mean(Evaluation["precision"].tolist()), 3)))
print("Recall: "+str(round(np.mean(Evaluation["recall"].tolist()), 3)))
print("F1 Score: "+str(round(np.mean(Evaluation["f1"].tolist()), 3)))
print("BLEU: "+str(round(np.mean(Evaluation["BLEU"].tolist()), 3)))
print("ROUGE-1: "+str(round(np.mean(Evaluation["ROUGE-1"].tolist()), 3)))
print("ROUGE-2: "+str(round(np.mean(Evaluation["ROUGE-2"].tolist()), 3)))
print("ROUGE-L: "+str(round(np.mean(Evaluation["ROUGE-L"].tolist()), 3)))
print("METEOR: "+str(round(np.mean(Evaluation["METEOR"].tolist()), 3)))
print("WER: "+str(round(np.mean(Evaluation["WER"].tolist()), 3)))

Accuracy: 0.902
Precision: 0.958
Recall: 0.95
F1 Score: 0.954
BLEU: 0.567
ROUGE-1: 0.89
ROUGE-2: 0.7
ROUGE-L: 0.861
METEOR: 0.849
WER: 0.175


In [68]:
pd.DataFrame(final_results).to_excel("../../Datasets/Word_Embedding_Results.xlsx", index=False)
pd.DataFrame(final_sentences).to_excel("../../Datasets/eval_dataset_pipeline1_bis.xlsx", index=False)
pd.DataFrame(Evaluation).to_excel("../../Datasets/eval_dataset_pipeline1_all_metrics.xlsx", index=False)